#  Network Security
## Background
Refer to [OSI Model](../computer-networks/osi_model.ipynb#osi_model)

## Security Weakness
The origin design of Internet did not take security into account.

Thus, below are the possible types of attacks that can be performed (derived from the [CIA Triad](./introduction.ipynb#cia-triad)):
* Interception: Unauthorized viewing
* Modification: Unauthorized change
* Fabrication: Unauthorized creation
* Interruption: Preventing of authorized access

At any layer of the OSI model, the attacker can modify **both the header and the data**.

## Name Resolution
Each entity has a name, which is different for each layer of the OSI model.

For instance: 
* at layer 7 (Application level), the entity is denoted by their url: "www.google.com"
* at layer 3 (Network level), the entity is denoted by their ip address: 74.125.200.99
* at layer 2 (Datalink level), the entity is denoted by their MAC address: 59:8c:c8:f3:8d:94

Thus, as information travel down the layers, they need a way of translating higher level addresses into lower level ones, through a **resolution protocol**.

During the design of resolution protocols, security was not accounted for, thus it is easy for attackers to influence the outcome of name resolutions.

With the background knowledge about [ARP](../computer-networks/osi_model.ipynb#arp), we will discuss possible vulnerabilities.


### Authentication in DNS
Each DNS query contains a 16-bit number known as the **Query ID (QID)**.
The response from the server also contains the same QID.
The requester only accepts the response if the QID matches.

Note that QID was not designed primarily for authentication, but rather as a way to find the DNS response that the requester is interested in.

Also, further note that there is no encryption or MAC involved in the protocol, thus **neither confidentiality nor integrity is guaranteed**.

### DNS Spoofing
The attacker can perform the following:
1. Victim makes a DNS query for a certain domain to find the IP address
2. Attacker see this query, and quickly replies with the IP address of his malicious server together with the corresponding QID
3. The DNS server sees the victim's query, and sends the DNS response
3. Because the attacker responded faster than the legitimate DNS server, the victim sees the malicious query first, and thus visits the malicious server instead of the genuine server

## Denial of Service (DoS)
DoS targets the availability of a service.

Types of DoS attacks:

|   | Terminating service | Exhausting resources |
| :---| :---- | :---- |
| Local Attacks  | Killing Processes | Spawning Processes |
|   | Crashing Processes | Occupying disk space |
|   | Reconfiguring System | |
| Remote Attacks  | Sending malformed packets | Packet flooding |

### Packet Flooding
The act of flooding the victim with requests or data.
Typically performed by amplifying small traffic into large traffic.

#### ICMP/Smurf floot attack
ICMP is a protocol typically used for network diagnosis.
The requester will send a **echo request** across the network, where each node in the network will reply to the request.
By viewing these **echo replies**, the requester can track which nodes are visitable to them.

However, this can be misused by attacker, as follows:

1. Attacker sends **ICMP PING** request to the router, telling the router to broadcast this request to all nodes.
The attacker forge the source IP using the victim's IP
2. The router will broadcast this echo request
3. Each node on the network that receives the request will send its echo reply to the source (which is spoofed to be the victim)

Thus, the attacker achieves an amplifying effect, where a single requests becomes amplified into a large quantity of requests to the victim.

However, do note that this attack is **usually not effective nowadays**, since most routers are configured not to broadcast these requests.

### Distributed Denial of Service (DDoS)
A straight forward way to take down a server is to flood it with requests.
However, it still requires a substantial number of attacker to take down a server through sheer numbers.

To gather this quantity of attacking entities, the attack can use viruses and trojan horses to infect a large number of devices.
These compromised machines are deemed as **bots/zombies**.
This collection of bot that the attacker has control over is called a **botnet/zombie army**.
This large quantity of machine serves as the resources need for the attacker to employ a DDoS attack.

## Attacking DNS servers
Note that attackers do not need to attack the target's server directly to prevent access.
They can target the DNS servers instead, so that people who wishes to visit the domain is unable to resolve to the correct addresses.
Thus, the DoS methods discussed can also be applied against the DNS servers instead.

# Protection
There are many security protocols that aims to achieve confidentiality and authenticity, however many of them work on different layers of the Internet.

As a general guideline, **if a security protocol protects layer $k$, it will protect information from layer $k$ and above from attackers who are at layer $k-1$ and below.**

For example, if a security mechanism protects layer 3, information from layer 4 and above are safe from attackers who are at layer 1 or 2.

However, note that it does not protect the layer 4 information from attackers at layer 4 and above, neither does it protect layer 1 or 2 information from any attackers.

One way to visualize this is to imagine an elevator which is meant to hold items.
The elevator starts at the top level and goes down until it reaches the ground floor, before coming up again.
At each level, some items will be placed into the elevator.

Right when the elevator descends to the level that the security mechanism acts on, it will wrap up all the items into a bag.
Note that no one can view the content inside the bag, however new content can be added into the elevator outside of the bag as the elevator descends further down.
And when the elevator is ascending, the bag will be open once the elevator returns to the floor the security mechanism is at.

Thus, the problem of which layer's information is protected depends on whether someone at a certain level can see that object (data) in the elevator.

## SSL/TLS
SSL/TLS works **right above** the transport layer (layer 4).

They achieves both confidentiality and authenticity.

## WPA
A protocol employed in home access WiFi.
Supercedes WEP, which is broken.

Provides protect at layer 1 and 2.
However, note that not all information at layer 2 is protected.

## IPsec
Provides authenticity/integrity of IP addresses, but does not provide confidentiality.

This means that attackers cannot spoof IP addresses, but can monitor the source and destination of network traffic.

Protects layer 3.

## Firewall
Having the above mechanism are insufficient to protect the network because certain services cannot be protected by them.
For example, DNS queries cannot be protected by SSL/TLS due to efficiency concerns.

Thus, firewalls acts as a control for the flow of traffic between networks (between untrusted public networks: Internet and trusted internal network).

Firewalls sits at the border between networks and monitor the traffic.
They control what traffic are allowed to enter/leave the network.

### Demilitarized Zone (DMZ)
A DMZ is a small sub-network that exposes an organization's external services to the Internet.
This would be services such as web servers, email servers or file transfer servers.

### Firewall rules
Firewall determines which traffic is allowed/disallowed via a set of rules, usually in the form of a table.
Each packet is compared against the rules in the table, from the top of the table downwards.
The first rule which matches the packet in question will be applied.
Thus, the most specific rule should be higher on the table, while the more general ones are lower.

For example, the following rule:

| Protocol | Source address | Destination address | Source port | Destination port | Action |
| :---: | :---: |:---: |:---: |:---: |:---: |
| HTTP | 192.168.1.1 | 170.32.43.* | * | 80 | Allow |

Note that "\*" is a wildcard that matches any values.

This rule allows HTTP request from `192.168.1.1` (any port) to any address of the form `170.32.43.*` at port 80.

By adding the below rule to the bottom of the table:

| Protocol | Source address | Destination address | Source port | Destination port | Action |
| :---: | :---: |:---: |:---: |:---: |:---: |
| * | * | * | * | * | Deny |

We can prevent all other sorts of traffic through the firewall.